# Dataloader

Package installation, loading, and dataloaders. There's also a simple model defined. You can change it your favourite architecture if you want.

In [ ]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

# Implement the Attacks

Functions are given a simple useful signature that you can start with. Feel free to extend the signature as you see fit.

You may find it useful to create a 'batched' version of PGD that you can use to create the adversarial attack.

In [ ]:
def pgd_untargeted(model, x, labels, k, eps, eps_step):
    model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach()
    adv_x.requires_grad_(True) 
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        # TODO: Calculate the loss
        loss = TODO
        loss.backward()
        # TODO: compute the adv_x
        # find delta, clamp with eps
   
    return adv_x

# Implement Adversarial Training

In [ ]:
def test_model_on_attacks(model, attack='pgd', eps=0.1):
    model.eval()
    tot_test, tot_acc = 0.0, 0.0
    for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        if attack == 'pgd':
            # TODO: get x_adv untargeted pgd
        elif attack == 'fgsm':
            # TODO: get x_adv using untargeted fgsm
        else:
            pass

        # get the testing accuracy and update tot_test and tot_acc
        tot_acc += TODO
        tot_test += TODO
            
    print('Robust accuracy %.5lf' % (tot_acc/tot_test), f'on {attack} attack with eps = {eps}')

In [ ]:
def train_model(model, num_epochs, enable_defense=True, attack='pgd', eps=0.1):
    learning_rate = 0.0001

    opt = optim.Adam(params=model.parameters(), lr=learning_rate)

    ce_loss = torch.nn.CrossEntropyLoss()

    tot_steps = 0

    for epoch in range(1,num_epochs+1):
        t1 = time.time()
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):

            if enable_defense:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                tot_steps += 1
                opt.zero_grad()
                if attack == 'pgd':
                    # TODO: get x_adv using untargeted pgd
                elif attack == 'fgsm':
                    # TODO: get x_adv using untargeted fgsm
                else:
                    pass
                # TODO: update model by calculating adversarial loss
                opt.step()
            else:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                tot_steps += 1
                opt.zero_grad()
                # TODO: update model by calculating clean loss
                opt.step()

        tot_test, tot_acc = 0.0, 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            # calculate the accuracy by updating tot_test and tot_acc
        t2 = time.time()

        print('Epoch %d: Accuracy %.5lf [%.2lf seconds]' % (epoch, tot_acc/tot_test, t2-t1))
        
        if enable_defense:
            test_model_on_attacks(model, 'pgd')

# Study Accuracy, Quality, etc.

Compare the various results and report your observations on the submission.

In [ ]:
## TODO: train the original model

In [ ]:
## TODO: PGD attack

In [ ]:
## TODO: PGD based adversarial training

In [ ]:
## TODO: test AT based training model


In [ ]:
## TODO: FGSM attack - targeted on AT